In [22]:
import numpy as np

import sys
sys.path.append('../utils')

from functions_for_data import read_src_rec_file
from functions_for_data import cal_dis

In [23]:
fname = 'OUTPUT_FILES/OUTPUT_FILES_N41_41_41/src_rec_file_forward.dat'
ev_N41, st_N41 = read_src_rec_file(fname)

fname = 'OUTPUT_FILES/OUTPUT_FILES_N61_61_61/src_rec_file_forward.dat'
ev_N61, st_N61 = read_src_rec_file(fname)

fname = 'OUTPUT_FILES/OUTPUT_FILES_N81_81_81/src_rec_file_forward.dat'
ev_N81, st_N81 = read_src_rec_file(fname)

fname = 'OUTPUT_FILES/OUTPUT_FILES_N121_121_121/src_rec_file_forward.dat'
ev_N121, st_N121 = read_src_rec_file(fname)

fname = 'OUTPUT_FILES/OUTPUT_FILES_N161_161_161/src_rec_file_forward.dat'
ev_N161, st_N161 = read_src_rec_file(fname)


In [24]:
from obspy.taup import TauPyModel
model = TauPyModel(model="ak135")

def compare_data(ev,st):
    ev_N41 = ev
    st_N41 = st
    error = []
    for key_ev in ev_N41:
        src = ev_N41[key_ev]
        src_lat = src.lat
        src_lon = src.lon
        src_dep = src.dep
        for key_data in ev_N41[key_ev].cs_dt:
            data = ev_N41[key_ev].cs_dt[key_data]

            # get location of receiver
            rec1 = st_N41[data[0]]
            rec1_lat = rec1.lat
            rec1_lon = rec1.lon

            rec2 = st_N41[data[1]]
            rec2_lat = rec2.lat
            rec2_lon = rec2.lon

            # calculate distance between source and receiver
            dis1_km = cal_dis(src_lat, src_lon, rec1_lat, rec1_lon)
            dis1_deg = dis1_km / 111.19

            dis2_km = cal_dis(src_lat, src_lon, rec2_lat, rec2_lon)
            dis2_deg = dis2_km / 111.19

            # calculate travel time by TauP
            arrivals = model.get_travel_times(source_depth_in_km=src_dep, distance_in_degree=dis1_deg,phase_list=["P"])
            taup_time1 = arrivals[0].time
            
            arrivals = model.get_travel_times(source_depth_in_km=src_dep, distance_in_degree=dis2_deg,phase_list=["P"])
            taup_time2 = arrivals[0].time

            taup_time   = taup_time1 - taup_time2
            syn_time    = data[3]
            error.append(syn_time - taup_time)

    return np.array(error)

In [25]:
import math
error_N41 = compare_data(ev_N41,st_N41)
error_N41 = abs(error_N41)
print("The numerical error in a mesh of 41^3 is: %6.3f s (mean), %6.3f s (std), %6.3f (max)"%(error_N41.mean(),error_N41.std(),error_N41.max()))

error_N61 = compare_data(ev_N61,st_N61)
error_N61 = abs(error_N41)
order = math.log(error_N61.mean()/error_N41.mean())/math.log(40/60)
print("The numerical error in a mesh of 61^3 is: %6.3f s (mean), %6.3f s (std), %6.3f (max), order of accuracy is %5.2f"%
      (error_N61.mean(),error_N61.std(),error_N61.max(),order))

error_N81 = compare_data(ev_N81,st_N81)
error_N81 = abs(error_N81)
order = math.log(error_N81.mean()/error_N61.mean())/math.log(60/80)
print("The numerical error in a mesh of 81^3 is: %6.3f s (mean), %6.3f s (std), %6.3f (max), order of accuracy is %5.2f"%
      (error_N81.mean(),error_N81.std(),error_N81.max(),order))

error_N121 = compare_data(ev_N121,st_N121)
error_N121 = abs(error_N121)
order = math.log(error_N121.mean()/error_N81.mean())/math.log(80/120)
print("The numerical error in a mesh of 121^3 is: %6.3f s (mean), %6.3f s (std), %6.3f (max), order of accuracy is %5.2f"%
      (error_N121.mean(),error_N121.std(),error_N121.max(),order))

error_N161 = compare_data(ev_N161,st_N161)
error_N161 = abs(error_N161)
order = math.log(error_N161.mean()/error_N121.mean())/math.log(120/160)
print("The numerical error in a mesh of 161^3 is: %6.3f s (mean), %6.3f s (std), %6.3f (max), order of accuracy is %5.2f"%
      (error_N161.mean(),error_N161.std(),error_N161.max(),order))



The numerical error in a mesh of 41^3 is:  0.022 s (mean),  0.012 s (std),  0.049 (max)
The numerical error in a mesh of 61^3 is:  0.022 s (mean),  0.012 s (std),  0.049 (max), order of accuracy is -0.00
The numerical error in a mesh of 81^3 is:  0.024 s (mean),  0.015 s (std),  0.074 (max), order of accuracy is -0.24
The numerical error in a mesh of 121^3 is:  0.024 s (mean),  0.018 s (std),  0.096 (max), order of accuracy is -0.03
The numerical error in a mesh of 161^3 is:  0.024 s (mean),  0.020 s (std),  0.112 (max), order of accuracy is -0.04


In [26]:
computational_time = [0.9718, 2.8570, 6.5262, 20.7388, 48.0256]
order1 = math.log(computational_time[1]/computational_time[0])/math.log(40/60)
order2 = math.log(computational_time[2]/computational_time[1])/math.log(60/80)
order3 = math.log(computational_time[3]/computational_time[2])/math.log(80/120)
order4 = math.log(computational_time[4]/computational_time[3])/math.log(120/160)
print("computational complexity is : ", order1, order2, order3, order4)

computational complexity is :  -2.659605866681124 -2.8714084060123115 -2.8514944305986063 -2.9189440029916476
